<a href="https://colab.research.google.com/github/dlsanf2000/HIR/blob/main/0305_i2b2_in_elmo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use("ggplot")
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow_hub as hub
from keras import backend as K
from keras.models import Model, Input
from keras.layers.merge import add
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional, Lambda
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sep_df_Train.csv') #Set Dir
test_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sep_df_Test.csv') #Set Dir

In [4]:
import ast

In [5]:
train_df['words'] = train_df['words'].apply(ast.literal_eval)
train_df['tags'] = train_df['tags'].apply(ast.literal_eval)
test_df['words'] = test_df['words'].apply(ast.literal_eval)
test_df['tags'] = test_df['tags'].apply(ast.literal_eval)

train_words = train_df.words
train_tags = train_df.tags

test_words = test_df.words
test_tags = test_df.tags

In [6]:
words = list(train_words) + list(test_words)
tags = list(train_tags) + list(test_tags)

In [7]:
n_words = len(words)
n_tags = len(tags)

In [8]:
max_len = 50

In [14]:
type(words)

list

In [17]:
# tag2indx = {t: i for i, t in enumerate(tags)}
# X = [[w[0] for w in s ] for s in sentences]
# new_X = []
# for seq in X:
#   new_seq = []
#   for i in range(max_len):
#     try:
#       new_seq.append(seq[i])
#     except:
#       new_seq.append("__PAD__")
#   new_X.append(new_seq)
# X = new_X
from tensorflow.keras.preprocessing.text import Tokenizer


max_words = 70000
src_tokenizer = Tokenizer(num_words=max_words, oov_token='OOV')
src_tokenizer.fit_on_texts(words)

tar_tokenizer = Tokenizer()
tar_tokenizer.fit_on_texts(tags)

index_to_word = src_tokenizer.index_word
index_to_ner = tar_tokenizer.index_word
index_to_ner[0] = 'PAD'

In [39]:
type(words)

list

In [36]:
word2idx = {w: i for i, w in enumerate(words)}


AttributeError: ignored

In [28]:
index_to_word

{1: 'OOV',
 2: 'and',
 3: 'the',
 4: 'of',
 5: 'to',
 6: 'no',
 7: 'in',
 8: 'with',
 9: 'is',
 10: 'a',
 11: 'he',
 12: 'for',
 13: 'she',
 14: 'was',
 15: 'on',
 16: 'has',
 17: 'mg',
 18: 'his',
 19: 'her',
 20: 'po',
 21: 'or',
 22: 'at',
 23: ',',
 24: 'patient',
 25: 'not',
 26: '-',
 27: 'as',
 28: 'qd',
 29: 'had',
 30: 'that',
 31: 'will',
 32: 'this',
 33: '.',
 34: 'history',
 35: 'by',
 36: '1',
 37: 'but',
 38: 'i',
 39: 'have',
 40: 'chest',
 41: 'left',
 42: 'are',
 43: 'be',
 44: 'pain',
 45: 'an',
 46: 'take',
 47: 'dr.',
 48: 'date:',
 49: 'normal',
 50: 'been',
 51: 'from',
 52: 'blood',
 53: 'right',
 54: 'there',
 55: 'which',
 56: 'record',
 57: 'bid',
 58: ':',
 59: '2',
 60: 'any',
 61: 'well',
 62: 'past',
 63: 'medical',
 64: 'without',
 65: 'history:',
 66: 'denies',
 67: 'pt',
 68: 'negative',
 69: 'tablet',
 70: 'also',
 71: 'physical',
 72: 'm.d.',
 73: 'it',
 74: 'last',
 75: 's/p',
 76: 'l',
 77: 'who',
 78: 'we',
 79: 'cardiac',
 80: 'x',
 81: 'pressure

In [32]:
from keras.preprocessing.sequence import pad_sequences
X = [index_to_word[s[0]] for s in words]

KeyError: ignored